# 13. Where, Filter
#### By Kwan Yin Andrew Chau

In [1]:
import nbfinder
from loader import g
from loader import __
from loader import P
from loader import sydneyVertexId
from loader import hkVertexId

## where()

__Same as simple has cases__

In [2]:
a = g.V().where(__.values('runways').is_(P.gt(5))).fold().next()
b = g.V().has('runways',P.gt(5)).fold().next()
assert a == b

__Airports with more than 60 unique routes from them__

In [3]:
g.V().hasLabel('airport').where(__.out('route').count().is_(P.gt(60))).count().next()

181

__Paths from Sydney where the destination has more than 3 runways and the distance is more than 4000__

In [4]:
g.V(sydneyVertexId).outE().and_(
    # more than 3 runways
    __.where(__.inV().has('runways', P.gt(3))),
    # distance more than 4000
    __.values('dist').is_(P.gt(4000))
).inV().path().by('code').by('dist').by('code').fold().next()

[['SYD', 7489, 'LAX'],
 ['SYD', 8574, 'DFW'],
 ['SYD', 7420, 'SFO'],
 ['SYD', 5073, 'HNL'],
 ['SYD', 4855, 'HND']]

__All the destinations from Sydney airport outside Australia__

In [5]:
a = g.V(sydneyVertexId).out().not_(__.where(__.in_('contains').has('code','AU'))).count().next()

g.V(sydneyVertexId).out().not_(
    # destination doesn't contain code AU
    __.where(__.in_('contains').has('code','AU'))
).valueMap('code','city', 'country').fold().next()

[{'country': ['US'], 'city': ['Los Angeles'], 'code': ['LAX']},
 {'country': ['US'], 'city': ['Dallas'], 'code': ['DFW']},
 {'country': ['JP'], 'city': ['Tokyo'], 'code': ['NRT']},
 {'country': ['US'], 'city': ['San Francisco'], 'code': ['SFO']},
 {'country': ['TH'], 'city': ['Bankok'], 'code': ['BKK']},
 {'country': ['AE'], 'city': ['Dubai'], 'code': ['DXB']},
 {'country': ['NZ'], 'city': ['Wellington'], 'code': ['WLG']},
 {'country': ['KR'], 'city': ['Seoul'], 'code': ['ICN']},
 {'country': ['US'], 'city': ['Honolulu'], 'code': ['HNL']},
 {'country': ['HK'], 'city': ['Hong Kong'], 'code': ['HKG']},
 {'country': ['SG'], 'city': ['Singapore'], 'code': ['SIN']},
 {'country': ['NZ'], 'city': ['Auckland'], 'code': ['AKL']},
 {'country': ['CN'], 'city': ['Beijing'], 'code': ['PEK']},
 {'country': ['NZ'], 'city': ['Christchurch'], 'code': ['CHC']},
 {'country': ['MY'], 'city': ['Kuala Lumpur'], 'code': ['KUL']},
 {'country': ['CA'], 'city': ['Vancouver'], 'code': ['YVR']},
 {'country': ['IN

Could have been done using has neq

In [6]:
b = g.V(sydneyVertexId).out().has('country', P.neq('AU')).count().next()
assert a == b

And using not has

In [7]:
c = g.V(sydneyVertexId).out().not_(__.has('country', 'AU')).count().next()
assert a == c

__You can use a special form of "and", "or" inside where__

Destinations out of HK that have more than 4 runways but not 6

In [8]:
g.V(hkVertexId).out().where(
    __.values('runways').is_(P.gt(4).and_(P.neq(6)))
).valueMap('code','runways').fold().next()

[{'runways': [7], 'code': ['DFW']},
 {'runways': [8], 'code': ['ORD']},
 {'runways': [5], 'code': ['YYZ']}]

Destinations out of HK that have 5 or 6 runways

In [9]:
g.V(hkVertexId).out().where(
    __.values('runways').is_(P.eq(5).or_(P.eq(6)))
).valueMap('code','runways').fold().next()

[{'runways': [6], 'code': ['BOS']},
 {'runways': [5], 'code': ['YYZ']},
 {'runways': [6], 'code': ['AMS']}]

__Destinations two flights away from HK that do not end up back in HK__

In [10]:
g.V(hkVertexId).as_('a').out().out().where(
    P.neq('a')
).path().by('code').limit(10).fold().next()

[['HKG', 'LAX', 'MCO'],
 ['HKG', 'LAX', 'FLL'],
 ['HKG', 'LAX', 'BWI'],
 ['HKG', 'LAX', 'ANC'],
 ['HKG', 'LAX', 'ATL'],
 ['HKG', 'LAX', 'AUS'],
 ['HKG', 'LAX', 'IAD'],
 ['HKG', 'LAX', 'RDU'],
 ['HKG', 'LAX', 'IAH'],
 ['HKG', 'LAX', 'MIA']]

__Using where() and by() together__

Find all the destinations you can fly to from Sydney that have the same number of runways Sydney does

In [11]:
g.V(sydneyVertexId).as_('a').out().where(
    P.eq('a')
).by('runways').valueMap('code','runways').fold().next()

[{'runways': [3], 'code': ['ICN']},
 {'runways': [3], 'code': ['SIN']},
 {'runways': [3], 'code': ['PEK']},
 {'runways': [3], 'code': ['CHC']},
 {'runways': [3], 'code': ['KUL']},
 {'runways': [3], 'code': ['YVR']},
 {'runways': [3], 'code': ['DEL']},
 {'runways': [3], 'code': ['LST']}]

## filter()
_Equivalent way using a filter_

In [12]:
g.V(sydneyVertexId).as_('a').out().as_('b').filter(
    __.select('a','b').by('runways').where('a', P.eq('b'))
).valueMap('code','runways').fold().next()

[{'runways': [3], 'code': ['ICN']},
 {'runways': [3], 'code': ['SIN']},
 {'runways': [3], 'code': ['PEK']},
 {'runways': [3], 'code': ['CHC']},
 {'runways': [3], 'code': ['KUL']},
 {'runways': [3], 'code': ['YVR']},
 {'runways': [3], 'code': ['DEL']},
 {'runways': [3], 'code': ['LST']}]

__Find all the destinations you can fly to from Sydney that have less runways than Sydney does__

In [13]:
g.V(sydneyVertexId).as_('a').out().as_('b').where(
    'a', P.gt('b')
).by('runways').valueMap('code','runways').fold().next()

[{'runways': [2], 'code': ['NRT']},
 {'runways': [2], 'code': ['BNE']},
 {'runways': [2], 'code': ['MEL']},
 {'runways': [2], 'code': ['BKK']},
 {'runways': [2], 'code': ['DXB']},
 {'runways': [1], 'code': ['WLG']},
 {'runways': [2], 'code': ['HKG']},
 {'runways': [2], 'code': ['AKL']},
 {'runways': [2], 'code': ['PER']},
 {'runways': [2], 'code': ['CBR']},
 {'runways': [2], 'code': ['PVG']},
 {'runways': [2], 'code': ['DRW']},
 {'runways': [2], 'code': ['SCL']},
 {'runways': [2], 'code': ['OOL']},
 {'runways': [2], 'code': ['JNB']},
 {'runways': [2], 'code': ['SGN']},
 {'runways': [1], 'code': ['AYQ']},
 {'runways': [2], 'code': ['DPS']},
 {'runways': [2], 'code': ['MNL']},
 {'runways': [2], 'code': ['DOH']},
 {'runways': [2], 'code': ['ADL']},
 {'runways': [2], 'code': ['AUH']},
 {'runways': [2], 'code': ['CGK']},
 {'runways': [1], 'code': ['NKG']},
 {'runways': [2], 'code': ['NAN']},
 {'runways': [2], 'code': ['ASP']},
 {'runways': [2], 'code': ['HAN']},
 {'runways': [2], 'code': ['

__Compare two different properties by adding a second "by" after "where__

Countries that have a contains edge into Sydney

In [14]:
g.V(sydneyVertexId).in_('contains').has('type', 'country').valueMap().fold().next()

[{'type': ['country'], 'code': ['AU'], 'desc': ['Australia']}]

We match the country from Airport, Sydney (a) with the code from the Country, Australia (b)

In [15]:
g.V(sydneyVertexId).as_('a','r').in_('contains').as_('b').where(
    'a', P.eq('b')
).by('country').by('code').select('a','r','b').by('code').by('region').by('desc').fold().next()

[{'a': 'SYD', 'r': 'AU-NSW', 'b': 'Australia'}]

Get the airports that are in the same region as Sydney is in

In [16]:
g.V(sydneyVertexId).values('region').as_('r').V().hasLabel('airport').as_('a').values('region').where(
    P.eq('r')
).by().local(__.select('a').values('city','code','region').fold()).fold().next()

[['SYD', 'Sydney', 'AU-NSW'],
 ['ABX', 'Albury', 'AU-NSW'],
 ['MYA', 'Moruya', 'AU-NSW'],
 ['PKE', 'Parkes', 'AU-NSW'],
 ['OAG', 'Orange', 'AU-NSW'],
 ['NRA', 'Narrandera', 'AU-NSW'],
 ['GFF', 'Griffith', 'AU-NSW'],
 ['NTL', 'Williamtown', 'AU-NSW'],
 ['ARM', 'Armidale', 'AU-NSW'],
 ['WGA', 'Wagga Wagga', 'AU-NSW'],
 ['MRZ', 'Moree', 'AU-NSW'],
 ['BHQ', 'Broken Hill', 'AU-NSW'],
 ['LSY', 'Lismore', 'AU-NSW'],
 ['TRO', 'Taree', 'AU-NSW'],
 ['BHS', 'Bathurst', 'AU-NSW'],
 ['NAA', 'Narrabri', 'AU-NSW'],
 ['LDH', 'Lord Howe Island', 'AU-NSW'],
 ['CFS', 'Coffs Harbour', 'AU-NSW'],
 ['BNK', 'Ballina', 'AU-NSW'],
 ['GFN', 'Grafton', 'AU-NSW'],
 ['DBO', 'Dubbo', 'AU-NSW'],
 ['OOL', 'Gold Coast', 'AU-NSW'],
 ['PQQ', 'Port Macquarie', 'AU-NSW'],
 ['MIM', 'Merimbula', 'AU-NSW'],
 ['TMW', 'Tamworth', 'AU-NSW'],
 ['BWU', 'Sydney', 'AU-NSW']]

In [17]:
g.V(sydneyVertexId).values('region').as_('r').V().hasLabel('airport').as_('a').values('region').where(
    P.eq('r')
).select('a').values('city','code','region').fold().next()

['SYD',
 'Sydney',
 'AU-NSW',
 'ABX',
 'Albury',
 'AU-NSW',
 'MYA',
 'Moruya',
 'AU-NSW',
 'PKE',
 'Parkes',
 'AU-NSW',
 'OAG',
 'Orange',
 'AU-NSW',
 'NRA',
 'Narrandera',
 'AU-NSW',
 'GFF',
 'Griffith',
 'AU-NSW',
 'NTL',
 'Williamtown',
 'AU-NSW',
 'ARM',
 'Armidale',
 'AU-NSW',
 'WGA',
 'Wagga Wagga',
 'AU-NSW',
 'MRZ',
 'Moree',
 'AU-NSW',
 'BHQ',
 'Broken Hill',
 'AU-NSW',
 'LSY',
 'Lismore',
 'AU-NSW',
 'TRO',
 'Taree',
 'AU-NSW',
 'BHS',
 'Bathurst',
 'AU-NSW',
 'NAA',
 'Narrabri',
 'AU-NSW',
 'LDH',
 'Lord Howe Island',
 'AU-NSW',
 'CFS',
 'Coffs Harbour',
 'AU-NSW',
 'BNK',
 'Ballina',
 'AU-NSW',
 'GFN',
 'Grafton',
 'AU-NSW',
 'DBO',
 'Dubbo',
 'AU-NSW',
 'OOL',
 'Gold Coast',
 'AU-NSW',
 'PQQ',
 'Port Macquarie',
 'AU-NSW',
 'MIM',
 'Merimbula',
 'AU-NSW',
 'TMW',
 'Tamworth',
 'AU-NSW',
 'BWU',
 'Sydney',
 'AU-NSW']